In [21]:
import h5py
from PIL import Image
import cv2
import numpy as np
import math
from tensorflow import keras
from tensorflow.keras import layers


In [178]:
#EDIT
filename = "C:/users/brend/Desktop/project3/MNIST_synthetic.h5"

f = h5py.File(filename,'r')

keys = list(f.keys())

testd = keys[0]
traind = keys[1]
trainl = keys[2]

test_data = list(f[testd])


train_data = list(f[traind])
train_labels = list(f[trainl])
f.close()

In [179]:

def find_digits(data):
    regions = []
    
    i = 0

    for dd in data:
#         im = Image.fromarray(dd[:,:,0])
#         im.show()

        sub_reg = []

        retval, threshold = cv2.threshold(dd, 1, 255, cv2.THRESH_BINARY)
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
        threshed = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, rect_kernel)
        contours, hierarchy = cv2.findContours(threshed, cv2.RETR_EXTERNAL,  
                                                         cv2.CHAIN_APPROX_SIMPLE)

        pts = []

        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)

            if h < 7 or h > 12 or len(cnt) == 1:
                continue

            pts.append((x,y))
        
        pts = sorted(pts, key=lambda x:x[0])
        
        for k in pts:
            x = math.floor(k[0])
            y = math.floor(k[1])

            d = dd[y-1:y+12, x-1:x+11, 0]
            
            if d.shape[0] != 13 or d.shape != 12:
                 d = np.resize(d, (13, 12))

            if not d.any():
                continue    
#             im = Image.fromarray(d)
#             im.show()
            sub_reg.append(d)

#         i += 1
#         if i == 5:
#             break
        regions.append(sub_reg)
    return regions




In [180]:
#check accuracy of number of digits found 


# print (len(tr_regs))
tr_regs = find_digits(train_data)
test_regs = find_digits(test_data)
lll = []
for t in tr_regs:
    lll.append(len(t))

ll = []
for t in train_labels:
    aa = []
    for z in t:
        if z != 10:
            aa.append(z)
    ll.append(len(aa))
    

b = 0

for i in range(len(lll)):
    if lll[i] != ll[i]:
        b += 1

print (b)
 
print (tr_regs[0][0].shape)

334
(13, 12)


In [181]:
#make data so can be trained by single region MNIST digit
x_train = []
y_train = []

i = 0
for t in tr_regs:
    j = 0
    for p in t:
        if j >= len(train_labels[i]) or train_labels[i][j] == 10:
            break
        x_train.append(p.astype("float32")/ 255) #normalize to [0,1]
        y_train.append(train_labels[i][j])
        j += 1
    i += 1

x_train = [np.expand_dims(np.array(xi),-1) for xi in x_train]
x_train = np.expand_dims(x_train, 0)
x_train = np.concatenate(x_train, axis=0)

# y_train = [] 
# for t in train_labels:
#     for n in t:
#         if n != 10:
#             y_train.append(n)
cats = []
for i in y_train:
    if i not in cats:
        cats.append(i)

print (x_train[0].shape)
print (x_train.shape)
print (len(y_train))
print (cats)
print (len(cats))

(13, 12, 1)
(167560, 13, 12, 1)
167560
[8, 5, 0, 3, 9, 7, 1, 2, 6, 4]
10


In [182]:
#inner model for classifying single digits

num_classes = 10
input_shape = (13, 12, 1)

# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
# x_train = x_train.astype("float32") / 255
# x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)

# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)
# print("x_train shape:", x_train.shape)
# print(x_train.shape[0], "train samples")
# print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

In [183]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 11, 10, 32)        320       
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 3, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 64)                0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)              

In [184]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
1179/1179 [==============================] - 9s 8ms/step - loss: 0.6797 - accuracy: 0.7982 - val_loss: 0.2355 - val_accuracy: 0.9372
Epoch 2/15
1179/1179 [==============================] - 10s 8ms/step - loss: 0.3245 - accuracy: 0.9100 - val_loss: 0.1635 - val_accuracy: 0.9573
Epoch 3/15
1179/1179 [==============================] - 9s 8ms/step - loss: 0.2664 - accuracy: 0.9275 - val_loss: 0.1356 - val_accuracy: 0.9644
Epoch 4/15
1179/1179 [==============================] - 10s 8ms/step - loss: 0.2344 - accuracy: 0.9365 - val_loss: 0.1151 - val_accuracy: 0.9709
Epoch 5/15
1179/1179 [==============================] - 10s 9ms/step - loss: 0.2161 - accuracy: 0.9427 - val_loss: 0.1122 - val_accuracy: 0.9724
Epoch 6/15
1179/1179 [==============================] - 11s 9ms/step - loss: 0.2026 - accuracy: 0.9462 - val_loss: 0.1016 - val_accuracy: 0.9755
Epoch 7/15
1179/1179 [==============================] - 11s 9ms/step - loss: 0.1928 - accuracy: 0.9494 - val_loss: 0.0996 - val_accu

In [161]:
preds = model.predict(x_train)

pp = []
for p in preds:
   pp.append(list(p).index(max(p))) 

yy = []
for y in y_train:
    yy.append(list(y).index(max(y)))

print (pp)
print (yy)
# score = model.evaluate(x_test, y_test, verbose=0)
# print("Test loss:", score[0])
# print("Test accuracy:", score[1])

[3, 0, 5, 8, 7, 5, 9, 8, 8, 9, 1, 9, 7, 1, 6, 2, 5, 2, 4, 3, 4, 1, 3, 1, 8, 4, 0, 4, 5, 3, 5, 3, 5, 4, 6, 1, 1, 2, 3, 9, 2, 7, 2, 2, 7, 7, 2, 0, 9, 0, 8, 2, 6, 4, 2, 7, 2, 0, 7, 9, 7, 2, 0, 1, 0, 4, 1, 5, 3, 8, 5, 8, 9, 5, 7, 3, 3, 9, 1, 6, 4, 9, 1, 2, 2, 7, 7, 5, 7, 7, 3, 8, 4, 3, 3, 1, 6, 3, 2, 4, 3, 3, 1, 9, 3, 3, 3, 1, 2, 1, 6, 0, 3, 5, 5, 9, 0, 4, 7, 0, 8, 0, 3, 6, 7, 2, 1, 3, 4, 9, 5, 5, 5, 5, 8, 3, 0, 5, 4, 2, 4, 7, 8, 3, 1, 8, 2, 1, 5, 5, 7, 3, 9, 9, 7, 7, 5, 4, 7, 0, 2, 3, 4, 0, 4, 9, 2, 3, 8, 4, 3, 6, 4, 7, 2, 3, 8, 6, 0, 3, 0, 7, 3, 1, 5, 8, 9, 4, 4, 9, 0, 0, 7, 5, 3, 8, 6, 8, 3, 6, 9, 1, 2, 9, 3, 8, 1, 1, 0, 6, 3, 6, 1, 6, 9, 0, 0, 1, 0, 6, 5, 1, 0, 8, 4, 4, 8, 2, 9, 4, 0, 9, 9, 2, 7, 8, 4, 3, 8, 3, 8, 5, 7, 4, 2, 1, 6, 3, 1, 2, 3, 9, 8, 7, 9, 2, 5, 3, 3, 4, 7, 5, 1, 0, 9, 2, 2, 1, 7, 7, 9, 6, 4, 5, 3, 4, 1, 2, 6, 8, 7, 4, 4, 4, 3, 1, 1, 5, 3, 1, 9, 8, 4, 3, 6, 8, 6, 8, 7, 9, 2, 5, 9, 8, 1, 1, 2, 1, 3, 8, 7, 0, 3, 7, 7, 7, 7, 4, 4, 7, 1, 0, 8, 7, 0, 7, 4, 1, 1, 7, 0, 5, 0, 